In [ ]:
import os
import whisper
import pandas as pd

In [ ]:
nas_dir = "/home/jaejoong/cocoanlab02"
edaic_dir = os.path.join(nas_dir, "projects/AIDA/test_dataset")

In [ ]:
model = whisper.load_model('large')

In [ ]:
edaic_csv = pd.read_csv(os.path.join(edaic_dir, "detailed_lables_mod.csv")) # 431,713: Wrong gender, 485,486: Wrong gender and age, 669,683: PCL-C sum omitted, All: Depression and PTSD criterion base on PHQ-8 >= 10 and PCL-C >= 45
edaic_csv = edaic_csv[edaic_csv["Participant"] >= 600].copy().reset_index(drop=True)

In [ ]:
for idx, row in edaic_csv.iterrows():

    print(f"\rWorking on Participant {row["Participant"]}...       ", end="", flush=True)

    audio_file = os.path.join(edaic_dir, "all_audio", "compnorm", str(row["Participant"]) + "_AUDIO.wav")
    result = model.transcribe(audio_file)

    rows = []
    for segment in result["segments"]:
        rows.append({
            "start_time": segment["start"],
            "stop_time": segment["end"],
            "speaker": "Ellie",
            "value": segment["text"].strip().lower()
        })
        
    df = pd.DataFrame(rows)
    ts_file = os.path.join(edaic_dir, "all_transcripts", "whisper", str(row["Participant"]) + "_WHISPER_AUTO_TRANSCRIPT.csv")
    df.to_csv(ts_file, index=False)